In [2]:
import requests
import pandas as pd

In [3]:
cards = pd.read_csv('commander_cards_all_v1.csv')

## Data preprocessing & cleaning

In [4]:
card_dict = cards.drop(columns=['double-sided'], axis=1) # we don't care if the cards are double-sided, for now

In [5]:
card_dict['card_name'] = card_dict['card_name'].str.split('/|\(', expand=True)[0] # remove all characters after ( or / etc

In [6]:
display(card_dict)

,img_url,card_name,cost
0,https://cards.scryfall.io/large/front/9/f/9f76...,"Eruth, Tormented Prophet",3
1,https://cards.scryfall.io/large/front/f/6/f6cd...,"Esika, God of the Tree",3
2,https://cards.scryfall.io/large/front/8/6/8695...,"Esior, Wardwing Familiar",2
3,https://cards.scryfall.io/large/front/2/e/2e39...,"Esix, Fractal Bloom",6
4,https://cards.scryfall.io/large/front/3/3/3340...,"Estrid, the Masked",4
...,...,...,...
1155,https://cards.scryfall.io/large/front/e/a/ea1a...,"Zaffai, Thunder Conductor",2
1156,https://cards.scryfall.io/large/front/a/c/ac81...,"Zagras, Thief of Heartbeats",2
1157,https://cards.scryfall.io/large/front/8/8/880f...,"Zahid, Djinn of the Lamp",2
1158,https://cards.scryfall.io/large/front/9/c/9c6d...,"Zalto, Fire Giant Duke",2


In [7]:
card_dict['card_name'] = card_dict['card_name'].str.replace('[, ]', '_', regex=True)

In [8]:
display(card_dict)

,img_url,card_name,cost
0,https://cards.scryfall.io/large/front/9/f/9f76...,Eruth__Tormented_Prophet_,3
1,https://cards.scryfall.io/large/front/f/6/f6cd...,Esika__God_of_the_Tree_,3
2,https://cards.scryfall.io/large/front/8/6/8695...,Esior__Wardwing_Familiar_,2
3,https://cards.scryfall.io/large/front/2/e/2e39...,Esix__Fractal_Bloom_,6
4,https://cards.scryfall.io/large/front/3/3/3340...,Estrid__the_Masked_,4
...,...,...,...
1155,https://cards.scryfall.io/large/front/e/a/ea1a...,Zaffai__Thunder_Conductor_,2
1156,https://cards.scryfall.io/large/front/a/c/ac81...,Zagras__Thief_of_Heartbeats_,2
1157,https://cards.scryfall.io/large/front/8/8/880f...,Zahid__Djinn_of_the_Lamp_,2
1158,https://cards.scryfall.io/large/front/9/c/9c6d...,Zalto__Fire_Giant_Duke_,2


In [9]:
print(card_dict.nunique()) # Check that the names are all unique - they are

img_url      1160
card_name    1160
cost           11
dtype: int64


In [41]:
dataset = card_dict.set_index('card_name')['img_url'].to_dict()

labels = card_dict.set_index('card_name')['cost'].to_dict()
print(labels)

{'Eruth__Tormented_Prophet_': 3, 'Esika__God_of_the_Tree_': 3, 'Esior__Wardwing_Familiar_': 2, 'Esix__Fractal_Bloom_': 6, 'Estrid__the_Masked_': 4, 'Etali__Primal_Conqueror_': 7, 'Etali__Primal_Storm_': 6, 'Etrata__Deadly_Fugitive_': 3, 'Etrata__the_Silencer_': 4, 'Eutropia_the_Twice-Favored_': 3, 'Evelyn__the_Covetous_': 5, 'Evra__Halcyon_Witness_': 6, 'Exava__Rakdos_Blood_Witch_': 4, 'Experiment_Kraj_': 6, 'Extus__Oriq_Overlord_': 4, 'Ezio_Auditore_da_Firenze_': 2, 'Ezio__Blade_of_Vengeance_': 5, 'Ezrim__Agency_Chief_': 5, 'Ezuri__Claw_of_Progress_': 4, 'Ezuri__Renegade_Leader_': 3, 'Ezuri__Stalker_of_Spheres_': 4, 'Faceless_One_': 5, 'Fain__the_Broker_': 3, 'Falco_Spara__Pactweaver_': 4, 'Faldorn__Dread_Wolf_Herald_': 3, 'Falthis__Shadowcat_Familiar_': 3, 'Fangorn__Tree_Shepherd_': 7, 'Faramir__Field_Commander_': 4, 'Faramir__Prince_of_Ithilien_': 4, 'Faramir__Steward_of_Gondor_': 3, "Farideh__Devil's_Chosen_": 4, 'Farid__Enterprising_Salvager_': 3, 'Farmer_Cotton_': 2, 'Far_Travele

In [42]:
def download_images(dataset, labels): # Downloading images from information given by a dictionary 
    for name, url in dataset.items():
        try:
            response = requests.get(url)
            if response.status_code == 200:
                # Save the image with a unique identifier, ie name
                with open(f'images/{name}{labels[name]}.jpg', 'wb') as file:
                    file.write(response.content)
                
                # Associate the label with the image
                mana_cost = labels.get(name)
                if mana_cost:
                    print(f"Downloaded image for card {name} with mana cost {mana_cost}")
                else:
                    print(f"Label not found for card {name}")
            else:
                print(f"Failed to download image for card {name}")
        except Exception as e:
            print(f"Error downloading image for card {name}: {str(e)}")

# NOTES
# Make card filename card_name_whatever_3, then reconstruct dataframe by splitting the name by the last underscore
            
# Turns out that this is more time efficient than the one I attempted to adapt below, because iterrows() is very slow



In [40]:
# Downloading images, adapted to dataframe instead of dictionaries
def download_images_dataframe(dataframe, limit=10):
    df_subset = dataframe.head(limit)  # Take the first 'limit' rows

    try:
        # Download images for the entire subset
        responses = [requests.get(url) for url in df_subset['img_url']]

        # Process each response
        for index, response in df_subset.iterrows():
            name = response['card_name']
            url = response['img_url']

            idx = index  # Index in the original DataFrame

            if responses[idx].status_code == 200:
                # Save the image with a unique identifier, i.e., name
                with open(f'images/{name}.jpg', 'wb') as file:
                    file.write(responses[idx].content)

                # Associate the label with the image
                mana_cost = response['cost']
                if pd.notna(mana_cost):
                    print(f"Downloaded image for card {name} with mana cost {mana_cost}")
                else:
                    print(f"Label not found for card {name}")
            else:
                print(f"Failed to download image for card {name}")

    except Exception as e:
        print(f"Error downloading images: {str(e)}")

In [43]:
download_images(dataset, labels)

Downloaded image for card Eruth__Tormented_Prophet_ with mana cost 3
Downloaded image for card Esika__God_of_the_Tree_ with mana cost 3
Downloaded image for card Esior__Wardwing_Familiar_ with mana cost 2
Downloaded image for card Esix__Fractal_Bloom_ with mana cost 6
Downloaded image for card Estrid__the_Masked_ with mana cost 4
Downloaded image for card Etali__Primal_Conqueror_ with mana cost 7
Downloaded image for card Etali__Primal_Storm_ with mana cost 6
Downloaded image for card Etrata__Deadly_Fugitive_ with mana cost 3
Downloaded image for card Etrata__the_Silencer_ with mana cost 4
Downloaded image for card Eutropia_the_Twice-Favored_ with mana cost 3
Downloaded image for card Evelyn__the_Covetous_ with mana cost 5
Downloaded image for card Evra__Halcyon_Witness_ with mana cost 6
Downloaded image for card Exava__Rakdos_Blood_Witch_ with mana cost 4
Downloaded image for card Experiment_Kraj_ with mana cost 6
Downloaded image for card Extus__Oriq_Overlord_ with mana cost 4
Downlo

Downloaded image for card Eruth__Tormented_Prophet_ with mana cost 3
Downloaded image for card Esika__God_of_the_Tree_ with mana cost 3
Downloaded image for card Esior__Wardwing_Familiar_ with mana cost 2
Downloaded image for card Esix__Fractal_Bloom_ with mana cost 6
Downloaded image for card Estrid__the_Masked_ with mana cost 4
Downloaded image for card Etali__Primal_Conqueror_ with mana cost 7
Downloaded image for card Etali__Primal_Storm_ with mana cost 6
Downloaded image for card Etrata__Deadly_Fugitive_ with mana cost 3
Downloaded image for card Etrata__the_Silencer_ with mana cost 4
Downloaded image for card Eutropia_the_Twice-Favored_ with mana cost 3
